In [1]:
import sys
sys.path.append("..")
from config import aws_key, aws_secret
from config import mongo_username, mongo_password, mongo_server_name
from config import postgres_server_name, postgres_database, postgres_username, postgres_port, postgres_password

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import tra_functions as tralib

In [5]:
db_url = f"postgres://{postgres_username}:{postgres_password}@{postgres_server_name}:{postgres_port}/{postgres_database}"
engine = create_engine(db_url)
conn = engine.connect()
session = Session(bind=engine)

# Crime

In [6]:
query = session.query(' * from all_crime')
crime_df = pd.read_sql(query.statement, engine)


In [7]:
crime_df['report_date'] = pd.to_datetime(crime_df['report_date'])
crime_df.head()

,Index,crime_type,report_date,Beat,Neigborhood,NPU,lat,lon
0,0,LARCENY-NON VEHICLE,2009-02-28,301.0,Adair Park,V,33.72710,-84.41360
1,1,LARCENY-NON VEHICLE,2009-02-28,509.0,Downtown,M,33.76158,-84.38496
2,2,LARCENY-NON VEHICLE,2009-02-28,210.0,Lenox,B,33.84676,-84.36212
3,3,LARCENY-NON VEHICLE,2009-02-28,204.0,Berkeley Park,D,33.80448,-84.41348
4,4,LARCENY-NON VEHICLE,2009-02-28,609.0,Edgewood,O,33.75674,-84.34619


In [8]:
def crime_variable (row):
    if row['crime_type'] == 'HOMICIDE':
        return 1
    if row['crime_type'] == 'ROBBERY-PEDESTRIAN':
        return 1
    if row['crime_type'] == 'BURGLARY-RESIDENCE':
        return 1
    if row['crime_type'] == 'AGG ASSAULT':
        return 1
    return 0

# applying the function to go through every row and create a new column called 'rainy'

crime_df['person_crime'] = crime_df.apply(lambda row:crime_variable(row),axis = 1)

In [9]:
crime_df['year'] = pd.DatetimeIndex(crime_df['report_date']).year
crime_df['month'] = pd.DatetimeIndex(crime_df['report_date']).month

In [10]:
crime_person_df = crime_df[["person_crime","year", "month"]]

In [11]:
crime_person_time_df = crime_person_df.groupby(["year", "month"]).sum().reset_index()
crime_person_time_df.head()

,year,month,person_crime
0,2009,1,1125
1,2009,2,777
2,2009,3,920
3,2009,4,944
4,2009,5,993


# Weather

In [12]:
query = session.query(' * from hist_weather')
atl_weather_df = pd.read_sql(query.statement, engine)


In [13]:
atl_weather_df['date_simple'] = pd.to_datetime(atl_weather_df['date_simple'])
atl_weather_df.head()

,id,date,date_iso,weather_id,weather_group,weather,weather_icon,temperature,temp_min,temp_max,feels_like,pressure,humidity,wind_speed,cloud_percentage,date_simple
0,0,1230768000,2009-01-01 00:00:00 +0000 UTC,800,Clear,sky is clear,01n,39.78,39.20,41.12,25.48,1023,40,16.11,1,2009-01-01
1,1,1230771600,2009-01-01 01:00:00 +0000 UTC,800,Clear,sky is clear,01n,39.02,37.40,41.49,30.45,1024,40,5.82,1,2009-01-01
2,2,1230775200,2009-01-01 02:00:00 +0000 UTC,800,Clear,sky is clear,01n,37.81,37.04,40.35,28.63,1025,42,6.93,1,2009-01-01
3,3,1230778800,2009-01-01 03:00:00 +0000 UTC,800,Clear,sky is clear,01n,35.82,35.55,35.96,27.25,1025,45,5.82,1,2009-01-01
4,4,1230782400,2009-01-01 04:00:00 +0000 UTC,800,Clear,sky is clear,01n,34.59,33.80,35.52,27.39,1025,47,3.36,1,2009-01-01


In [14]:
atl_weather_df['year'] = pd.DatetimeIndex(atl_weather_df['date_simple']).year
atl_weather_df['month'] = pd.DatetimeIndex(atl_weather_df['date_simple']).month

In [15]:
def weather_variable (row):
    if row['weather_group'] == 'Rain':
        return 1
    if row['weather_group'] == 'Thunderstorm':
        return 1
    if row['weather_group'] == 'Snow':
        return 1
    return 0

# applying the function to go through every row and create a new column called 'rainy'

atl_weather_df['percentage_rain'] = atl_weather_df.apply(lambda row:weather_variable(row),axis = 1)

In [16]:
real_feel_Time_df = atl_weather_df[["feels_like","year", "month"]]
real_feel_TimeGroup_df = real_feel_Time_df.groupby(["year", "month"]).mean().reset_index()
real_feel_TimeGroup_df.head()

,year,month,feels_like
0,2009,1,35.386667
1,2009,2,38.375982
2,2009,3,48.506102
3,2009,4,55.358583
4,2009,5,68.471331


In [17]:
atl_weather_df['date_simple'].min()

Timestamp('2009-01-01 00:00:00')

In [18]:
atl_weather_df['date_simple'].max()

Timestamp('2020-03-31 00:00:00')

# Individual Data

# Average Real Feel by Month

In [19]:
real_feel_TimeGroup_df["day"] = 1

In [20]:
real_feel_TimeGroup_df["date"] = pd.to_datetime(real_feel_TimeGroup_df[['year', 'month', 'day']])

In [21]:
# Added by Kannan.  Change date type
real_feel_TimeGroup_df["date"]= real_feel_TimeGroup_df["date"].astype(str)

In [ ]:
# Commented by Kannan
# real_feel_time_df[(real_feel_time_df['date'] > '2009-01-01') & (real_feel_time_df['date'] < '2020-03-31')]

In [23]:
# Un-commented by Kannan
real_feel_time_df = real_feel_TimeGroup_df[["date","feels_like"]]
real_feel_time_df = real_feel_time_df.set_index("date")
real_feel_time_df = real_feel_time_df.loc['2009-01-01':'2020-03-31']

In [24]:
real_feel_time_df.head()

,feels_like
date,
2009-01-01,35.386667
2009-02-01,38.375982
2009-03-01,48.506102
2009-04-01,55.358583
2009-05-01,68.471331


In [25]:
len(real_feel_time_df)

135

In [27]:
tralib.write_df_to_mongo_as_json("real_feel_over_time", real_feel_time_df)

# Average Average Temp by Month

In [ ]:
temperature_brokenDate_df = atl_weather_df[["temperature","year", "month"]]

In [ ]:
temperature_brokenDate_df = temperature_brokenDate_df.groupby(["year", "month"]).mean().reset_index()
temperature_brokenDate_df.head()

In [ ]:
temperature_brokenDate_df["day"] = 1

In [ ]:
temperature_brokenDate_df["date"] = pd.to_datetime(temperature_brokenDate_df[['year', 'month', 'day']])

In [ ]:
temperature_time_df = temperature_brokenDate_df[["date","temperature"]]
temperature_time_df  = temperature_time_df.set_index("date")

In [ ]:
temp_over_time = temperature_time_df.loc['2009-01-01':'2020-03-31']

In [ ]:
temp_over_time.head()

In [ ]:
len(temp_over_time)

In [ ]:
tralib.write_df_to_mongo_as_json("temp_over_time", temp_over_time)

# Average Rain Percentage by Month

In [ ]:
rain_brokenDate_df = atl_weather_df[["percentage_rain","year", "month"]]

In [ ]:
rain_brokenDate_df = rain_brokenDate_df.groupby(["year", "month"]).mean().reset_index()
rain_brokenDate_df.head()

In [ ]:
rain_brokenDate_df["day"] = 1

In [ ]:
rain_brokenDate_df["date"] = pd.to_datetime(rain_brokenDate_df[['year', 'month', 'day']])

In [ ]:
rain_over_time_df = rain_brokenDate_df[["date","percentage_rain"]]
rain_over_time_df  = rain_over_time_df.set_index("date")

In [ ]:
rain_over_time = rain_over_time_df.loc['2009-01-01':'2020-03-31']

In [ ]:
rain_over_time.head()

In [ ]:
len(rain_over_time)

In [ ]:
tralib.write_df_to_mongo_as_json("rain_over_time", rain_over_time)

# Average Cloud level by month

In [ ]:
cloud_brokenDate_df = atl_weather_df[["cloud_percentage","year", "month"]]

In [ ]:
cloud_brokenDate_df = cloud_brokenDate_df.groupby(["year", "month"]).mean().reset_index()
cloud_brokenDate_df.head()

In [ ]:
cloud_brokenDate_df["day"] = 1

In [ ]:
cloud_brokenDate_df["date"] = pd.to_datetime(cloud_brokenDate_df[['year', 'month', 'day']])

In [ ]:
cloud_over_time_df = cloud_brokenDate_df[["date","cloud_percentage"]]
cloud_over_time_df  = cloud_over_time_df.set_index("date")

In [ ]:
cloud_over_time = cloud_over_time_df.loc['2009-01-01':'2020-03-31']

In [ ]:
cloud_over_time.head()

In [ ]:
len(cloud_over_time)

In [ ]:
tralib.write_df_to_mongo_as_json("cloud_over_time", cloud_over_time)

# Personal Crimes by Month

In [ ]:
crime_person_time_df.head()

In [ ]:
crime_person_time_df["day"] = 1

In [ ]:
crime_person_time_df["date"] = pd.to_datetime(crime_person_time_df[['year', 'month', 'day']])

In [ ]:
personal_crime_df = crime_person_time_df[["date","person_crime"]]
personal_crime_df  = personal_crime_df.set_index("date")

In [ ]:
personal_crime_over_time = personal_crime_df.loc['2009-01-01':'2020-03-31']

In [ ]:
personal_crime_over_time.head()

In [ ]:
len(personal_crime_over_time)

In [ ]:
tralib.write_df_to_mongo_as_json("personal_crime_over_time", personal_crime_over_time)

# All Crime by Month

In [ ]:
all_crime_df = crime_df[["crime_type","year", "month"]]
all_crime_df.head()

In [ ]:
all_crime_time_df = all_crime_df.groupby(["year", "month"]).count().reset_index()
all_crime_time_df.head()

In [ ]:
all_crime_time_df["day"] = 1

In [ ]:
all_crime_time_df["date"] = pd.to_datetime(all_crime_time_df[['year', 'month', 'day']])

In [ ]:
all_crime_df = all_crime_time_df[["date","crime_type"]]
all_crime_df  = all_crime_df.set_index("date")

In [ ]:
all_crime_over_time = all_crime_df.loc['2009-01-01':'2020-03-31']

In [ ]:
all_crime_over_time.head()

In [ ]:
len(all_crime_over_time)

In [ ]:
tralib.write_df_to_mongo_as_json("all_crime_over_time", all_crime_over_time)

# Join Crime and Weather

In [ ]:
personCrime_temp_df = pd.merge(crime_person_time_df,real_feel_TimeGroup_df, how = 'inner', on = ['year','month'])

In [ ]:
personCrime_temp_df["day"] = 1

In [ ]:
personCrime_temp_df["date"] = pd.to_datetime(personCrime_temp_df[['year', 'month', 'day']])

In [ ]:
org_crime_temp_df = personCrime_temp_df[["date","person_crime","feels_like"]]
org_crime_temp_df = org_crime_temp_df.set_index("date")

In [ ]:
# write to mongo
import tra_functions as tralib
tralib.write_df_to_mongo_as_json("Ethan_Crime_Weather_Exploration", org_crime_temp_df)

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Date')
ax1.set_ylabel('Count Crimes', color=color)
ax1.plot(org_crime_temp_df.index, org_crime_temp_df.person_crime, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Real Feel', color=color)  # we already handled the x-label with ax1
ax2.plot(org_crime_temp_df.index, org_crime_temp_df.feels_like, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()